<img src="https://nyp-aicourse.s3.ap-southeast-1.amazonaws.com/agods/nyp_ago_logo.png" width='400'/>

# Fraud Detection 

In this exercise, we will build a financial fraud detection model. The model is a binary classifier that classifies a transaction as non-fraud (negative case) and fraud (positive case).

There is a lack of public available datasets on financial services and specially in the emerging mobile money transactions domain. We will be using a sythetic dataset called PaySim. PaySim simulates mobile money transactions based on a sample of real transactions extracted from one month of financial logs from a mobile money service implemented in an African country. 

Here are the description of the different columns of the PaySim dataset: 

|Field|Description|
|-----|-----|
|step|Maps a unit of time in the real world. In this case 1 step is 1 hour of time. Total steps 744 (30 days simulation).|
|type|CASH-IN, CASH-OUT, DEBIT, PAYMENT and TRANSFER|
|amount|Amount of the transaction in local currency|
|nameOrig|Customer who started the transaction|
|oldbalanceOrg|Initial balance before the transaction|
|newbalanceOrig|New balance after the transaction|
|nameDest|Customer who is the recipient of the transaction|
|oldbalanceDest|Initial balance recipient before the transaction. Note that there is not information for customers that start with M (Merchants)|
|newbalanceDest|New balance recipient after the transaction. Note that there is not information for customers that start with M (Merchants)|
|isFlaggedFraud|The business model aims to control massive transfers from one account to another and flags illegal attempts. An illegal attempt in this dataset is an attempt to transfer more than 200.000 in a single transaction|
|isFraud|This is the transactions made by the fraudulent agents inside the simulation. In this specific dataset the fraudulent behavior of the agents aims to profit by taking control or customers accounts and try to empty the funds by transferring to another account and then cashing out of the system|


## Import Packages

In [65]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.lines as mlines
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import average_precision_score
from sklearn.metrics import classification_report
from xgboost.sklearn import XGBClassifier
from xgboost import plot_importance, to_graphviz

In [2]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

## Import the Data 

In [3]:
df = pd.read_csv('Fraud.csv')

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6362620 entries, 0 to 6362619
Data columns (total 11 columns):
 #   Column          Dtype  
---  ------          -----  
 0   step            int64  
 1   type            object 
 2   amount          float64
 3   nameOrig        object 
 4   oldbalanceOrg   float64
 5   newbalanceOrig  float64
 6   nameDest        object 
 7   oldbalanceDest  float64
 8   newbalanceDest  float64
 9   isFraud         int64  
 10  isFlaggedFraud  int64  
dtypes: float64(5), int64(3), object(3)
memory usage: 534.0+ MB


For consistency, let's correct spelling of original column headers.

In [5]:
df = df.rename(columns={'oldbalanceOrg':'oldBalanceOrig', 'newbalanceOrig':'newBalanceOrig', \
                        'oldbalanceDest':'oldBalanceDest', 'newbalanceDest':'newBalanceDest'})
df.head()

,step,type,amount,nameOrig,oldBalanceOrig,newBalanceOrig,nameDest,oldBalanceDest,newBalanceDest,isFraud,isFlaggedFraud
0,1,PAYMENT,9839.64,C1231006815,170136.0,160296.36,M1979787155,0.0,0.0,0,0
1,1,PAYMENT,1864.28,C1666544295,21249.0,19384.72,M2044282225,0.0,0.0,0,0
2,1,TRANSFER,181.00,C1305486145,181.0,0.00,C553264065,0.0,0.0,1,0
3,1,CASH_OUT,181.00,C840083671,181.0,0.00,C38997010,21182.0,0.0,1,0
4,1,PAYMENT,11668.14,C2048537720,41554.0,29885.86,M1230701703,0.0,0.0,0,0


Let's check for any missing values. It turns out there are no obvious missing values but, as we will see below, value 0 may be used as a proxy if no data is available (e.g. those in the newBalanaceOrig, newBalanceDest, etc)

In [6]:
df.isna().sum()

step              0
type              0
amount            0
nameOrig          0
oldBalanceOrig    0
newBalanceOrig    0
nameDest          0
oldBalanceDest    0
newBalanceDest    0
isFraud           0
isFlaggedFraud    0
dtype: int64

## Exploratory Data Analysis

In this section, we will do some data-wrangling to gain more insights into the dataset.

**Exercise** 

Let's find out how many different types of transactions first. 

In [7]:
## complete the code

print(df.type.value_counts())
print(df.type.unique())

CASH_OUT    2237500
PAYMENT     2151495
CASH_IN     1399284
TRANSFER     532909
DEBIT         41432
Name: type, dtype: int64
['PAYMENT' 'TRANSFER' 'CASH_OUT' 'DEBIT' 'CASH_IN']


#### Which types of transactions are fraudulent? 

We would like to find out which type of transactions are most fraudulent? We find that of the five types of transactions, fraud occurs only in two of them:
'TRANSFER' where money is sent to a customer / fraudster and 'CASH_OUT' where money is sent to a merchant who pays the customer / fraudster in cash. 

In [8]:
df.loc[df.isFraud == 1].type.unique()

array(['TRANSFER', 'CASH_OUT'], dtype=object)

Let's find out the percentage of TRANSFER transactions that are fraudulent. 

In [9]:
num_fraudulent_xfer = len(df.loc[ (df.isFraud == 1) & (df.type == 'TRANSFER')])
percentage_fraudulent_xfer = num_fraudulent_xfer / len(df.loc[df.type == 'TRANSFER'])
print(f'number of fraudulent TRANSFER = {num_fraudulent_xfer}')
print(f'percentage of fraudulent TRANSFER = {percentage_fraudulent_xfer}')

number of fraudulent TRANSFER = 4097
percentage of fraudulent TRANSFER = 0.007687991758442811


**Exercise**

1. Find out how many fraudulent CASH_OUT?  
2. Find out what is the percentage of CASH_OUT that is fraudulent.
3. What do you observe? 

<details><summary>Click here for solution</summary>
<br/>
    
```python 
num_fraudulent_cash_out = len(df.loc[ (df.isFraud == 1) & (df.type == 'CASH_OUT')])
percentage_fraudulent_cash_out = num_fraudulent_cash_out / len(df.loc[df.type == 'CASH_OUT'])
print(f'number of fraudulent CASH_OUT = {num_fraudulent_cash_out}')
print(f'percentage of fraudulent CASH_OUT = {percentage_fraudulent_cash_out}')
```
<br/>
Remarkably, the number of fraudulent TRANSFERs almost equals the number of fraudulent CASH_OUTs These observations appear, at first, to bear out the description provided on Kaggle for the modus operandi of fraudulent transactions in  this dataset, namely, fraud is committed by first transferring out funds to another account which subsequently cashes it out.


In [10]:
## complete the code 

num_fraudulent_cash_out = len(df.loc[ (df.isFraud == 1) & (df.type == 'CASH_OUT')])
percentage_fraudulent_cash_out = num_fraudulent_cash_out / len(df.loc[df.type == 'CASH_OUT'])
print(f'number of fraudulent CASH_OUT = {num_fraudulent_cash_out}')
print(f'percentage of fraudulent CASH_OUT = {percentage_fraudulent_cash_out}')

number of fraudulent CASH_OUT = 4116
percentage of fraudulent CASH_OUT = 0.0018395530726256983


#### Are there account labels common to fraudulent TRANSFERs and CASH_OUTs?

From the data description, the modus operandi for committing fraud involves first making a TRANSFER to a (fraudulent) account which in turn conducts a CASH_OUT. CASH_OUT involves transacting with a merchant who pays out cash. Thus, within this two-step process, the fraudulent account would be both, the destination in a TRANSFER and the originator in a CASH_OUT.

In [11]:
dfFraudTransfer = df.loc[(df.isFraud == 1) & (df.type == 'TRANSFER')]
dfFraudCashout = df.loc[(df.isFraud == 1) & (df.type == 'CASH_OUT')]

print('\nOf all fraudulent transactions, destinations for TRANSFERS are also originators for CASH_OUTs? {}'.format(\
(dfFraudTransfer.nameDest.isin(dfFraudCashout.nameOrig)).any())) # False


Of all fraudulent transactions, destinations for TRANSFERS are also originators for CASH_OUTs? False


However, the data shows above that there are no  such common accounts among fraudulent transactions. Thus, the data is not imprinted with the expected modus-operandi.

#### Are the destination accounts with zero balances before and after non-zero amount is transacted normal?  

The data has several transactions with zero balances in the destination account both before and after a non-zero amount is transacted. 

Let's find out the how many of these transactions, of type TRANSFER/CASH_OUT,  are actually fraudulent. 

In [12]:
dfFraudTransferCashOut = df.loc[(df.isFraud == 1) & ((df.type == 'TRANSFER') | (df.type == 'CASH_OUT'))]
dfNonFraudTransferCashOut = df.loc[(df.isFraud == 0) & ((df.type == 'TRANSFER') | (df.type == 'CASH_OUT'))]

print('\nThe fraction of fraudulent transactions with \'oldBalanceDest\' = \
\'newBalanceDest\' = 0 although the transacted \'amount\' is non-zero is: {}'.\
format(len(dfFraudTransferCashOut.loc[(dfFraudTransferCashOut.oldBalanceDest == 0) & \
(dfFraudTransferCashOut.newBalanceDest == 0) & (dfFraudTransferCashOut.amount != 0)]) / (1.0 * len(dfFraudTransferCashOut))))


The fraction of fraudulent transactions with 'oldBalanceDest' = 'newBalanceDest' = 0 although the transacted 'amount' is non-zero is: 0.4955558261293072


**Exercise** 

Now find out how many of these transactions (of type TRANSFER/CASH_OUT) are genuine (non-fraudulent) transaction. 

<details><summary>Click here for solution</summary>
<br/>

```python 
print('\nThe fraction of genuine transactions with \'oldBalanceDest\' = \
newBalanceDest\' = 0 although the transacted \'amount\' is non-zero is: {}'.\
format(len(dfNonFraudTransferCashOut.loc[(dfNonFraudTransferCashOut.oldBalanceDest == 0) & \
(dfNonFraudTransferCashOut.newBalanceDest == 0) & (dfNonFraudTransferCashOut.amount != 0 )]) / (1.0 * len(dfNonFraudTransferCashOut))))
```
<br/>
</details>

In [13]:
## Complete the code 

print('\nThe fraction of genuine transactions with \'oldBalanceDest\' = \
newBalanceDest\' = 0 although the transacted \'amount\' is non-zero is: {}'.\
format(len(dfNonFraudTransferCashOut.loc[(dfNonFraudTransferCashOut.oldBalanceDest == 0) & \
(dfNonFraudTransferCashOut.newBalanceDest == 0) & (dfNonFraudTransferCashOut.amount != 0 )]) / (1.0 * len(dfNonFraudTransferCashOut))))



The fraction of genuine transactions with 'oldBalanceDest' = newBalanceDest' = 0 although the transacted 'amount' is non-zero is: 0.0006176245277308345


The fraction of such transactions, where zero likely denotes a missing value, is much larger in fraudulent (50%) compared to genuine transactions (0.06%). This shows that a 0 in the oldBalanceDest and newBalanceDest is a strong indicator of fraud.

In [18]:
print('\nThe fraction of fraudulent transactions with \'oldBalanceOrig\' = \
\'newBalanceOrig\' = 0 although the transacted \'amount\' is non-zero is: {}'.\
format(len(dfFraudTransferCashOut.loc[(dfFraudTransferCashOut.oldBalanceOrig == 0) & \
(dfFraudTransferCashOut.newBalanceOrig == 0) & (dfFraudTransferCashOut.amount != 0)]) / (1.0 * len(dfFraudTransferCashOut))))

print('\nThe fraction of genuine transactions with \'oldBalanceOrig\' = \
newBalanceOrig\' = 0 although the transacted \'amount\' is non-zero is: {}'.\
format(len(dfNonFraudTransferCashOut.loc[(dfNonFraudTransferCashOut.oldBalanceOrig == 0) & \
(dfNonFraudTransferCashOut.newBalanceOrig == 0) & (dfNonFraudTransferCashOut.amount != 0 )]) / (1.0 * len(dfNonFraudTransferCashOut))))


The fraction of fraudulent transactions with 'oldBalanceOrig' = 'newBalanceOrig' = 0 although the transacted 'amount' is non-zero is: 0.0030439547059539756

The fraction of genuine transactions with 'oldBalanceOrig' = newBalanceOrig' = 0 although the transacted 'amount' is non-zero is: 0.4737321319703598


### Feature-engineering

Motivated by the possibility of zero-balances serving to differentiate between fraudulent and genuine transactions, we create 2 new features (columns) recording errors in the  originating and destination accounts for each transaction.

In [23]:
df['errorBalanceOrig'] = df.newBalanceOrig + df.amount - df.oldBalanceOrig
df['errorBalanceDest'] = df.oldBalanceDest + df.amount - df.newBalanceDest

In [26]:
df.columns

Index(['step', 'type', 'amount', 'nameOrig', 'oldBalanceOrig',
       'newBalanceOrig', 'nameDest', 'oldBalanceDest', 'newBalanceDest',
       'isFraud', 'isFlaggedFraud', 'errorBalanceOrig', 'errorBalanceDest'],
      dtype='object')

#### Correlation Matrix 

Let's find out the correlation of each of our numerical features with the target label, for all TRANSFER/CASH_OUT transactions.

In [47]:
dfTransferCashOut = df[ (df.type == 'TRANSFER') | (df.type == 'CASH_OUT') ] 

In [48]:
corr_matrix = dfTransferCashOut.corr()
# corr_matrix['median_house_value'].sort_values(ascending=False)

C:\Users\markk\AppData\Local\Temp\ipykernel_22296\2914629405.py:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  corr_matrix = dfTransferCashOut.corr()


In [49]:
corr_matrix['isFraud'].abs().sort_values(ascending=False)

isFraud             1.000000
oldBalanceOrig      0.347582
amount              0.070660
errorBalanceDest    0.069935
newBalanceOrig      0.063557
step                0.048671
isFlaggedFraud      0.044072
errorBalanceOrig    0.017149
oldBalanceDest      0.014960
newBalanceDest      0.008978
Name: isFraud, dtype: float64

## Create Train/Test Set 

From the exploratory data analysis (EDA), we know that fraud only occurs in 'TRANSFER's and 'CASH_OUT's. So we create a train/test set only from those transaction. Also, we will drop the nameOrig and nameDest, as our EDA shows that they are not relevant in predicting if a transaction is fraud or not.  We also need to convert the TRANSFER and CASHOUT to a numeric value instead.

**Exercise**

1. Create a dataframe that consists of TRANSFER/CASH_OUT transactions only
2. Drop the following features 'nameOrig', 'nameDest' 
3. Map the type TRANSFER to numeric value 0, and CASH_OUT to numeric value 1
4. create features (X), and labels (y) 
5. create train/test split of 80:20 ratio

In [61]:
## Complete the code 

# 1. Create a dataframe that consists of TRANSFER/CASH_OUT transactions only

dfTransferCashOut = df.loc[(df.type == 'TRANSFER') | (df.type == 'CASH_OUT')]

# 2. Drop the following features 'nameOrig', 'nameDest'
dfTransferCashOut = dfTransferCashOut.drop(['nameOrig', 'nameDest', 'isFlaggedFraud'], axis = 1)

# 3. Map the type TRANSFER to numeric value 0, and CASH_OUT to numeric value 1
dfTransferCashOut['type'] = dfTransferCashOut['type'].apply(lambda x: 0 if x == 'TRANSFER' else 1)

# 4. create features (X) and labels (y) 
y = dfTransferCashOut['isFraud'] 
X = dfTransferCashOut.drop('isFraud', axis=1)

# 5. Create train/test split of 80:20 ratio 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 49)

In [62]:
X_train.head()

,step,type,amount,oldBalanceOrig,newBalanceOrig,oldBalanceDest,newBalanceDest,errorBalanceOrig,errorBalanceDest
4702850,331,1,143921.86,0.0,0.0,297326.54,697212.70,143921.86,-255964.30
2692898,211,1,86329.28,21375.0,0.0,2295.48,88624.76,64954.28,0.00
1654445,158,1,174981.93,0.0,0.0,188724.40,363706.32,174981.93,0.01
6354398,709,1,268789.99,873.0,0.0,1786009.49,2054799.48,267916.99,0.00
1394097,139,1,77237.55,543.0,0.0,404023.38,481260.93,76694.55,0.00


## Modeling 

We can see from below, the data is highly imbalanced. 

In [63]:
y.value_counts()

0    2762196
1       8213
Name: isFraud, dtype: int64

As the data is highly imbalanced, we will use the Area under Precision/Recall Curve (Average Precision) as metrics to measure the performance of the classifier.

#### Linear Models

Let's train a Logistic Regressor to classify the fraud.

In [64]:
from sklearn.linear_model import LogisticRegression 

lr_clf = LogisticRegression()
lr_clf.fit(X_train, y_train)

LogisticRegression()

In [78]:
test_probs = lr_clf.predict_proba(X_test)
print('AUPRC for test set = {}'.format(average_precision_score(y_test, test_probs[:, 1])))

AUPRC for test set = 0.5185221436101777


In [79]:
train_probs = lr_clf.predict_proba(X_train)
print('AUPRC for train set = {}'.format(average_precision_score(y_train, train_probs[:, 1])))

AUPRC for train set = 0.5208191479652762


In [76]:
test_preds = lr_clf.predict(X_test)
print(classification_report(y_test, test_preds))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00    552386
           1       0.48      0.42      0.45      1696

    accuracy                           1.00    554082
   macro avg       0.74      0.71      0.72    554082
weighted avg       1.00      1.00      1.00    554082



The performance is not great. Our recall/precison for fraud class is rather disappointing.  Let's apply weights to the class and try again. 

In [77]:
lr_clf = LogisticRegression(class_weight='balanced')
lr_clf.fit(X_train, y_train)

LogisticRegression(class_weight='balanced')

In [80]:
train_probs = lr_clf.predict_proba(X_train)
print('AUPRC for train set = {}'.format(average_precision_score(y_train, train_probs[:, 1])))
test_probs = lr_clf.predict_proba(X_test)
print('AUPRC for test set = {}'.format(average_precision_score(y_test, test_probs[:, 1])))

AUPRC for train set = 0.5208191479652762
AUPRC for test set = 0.5185221436101777


In [81]:
test_preds = lr_clf.predict(X_test)
print(classification_report(y_test, test_preds))

              precision    recall  f1-score   support

           0       1.00      0.88      0.94    552386
           1       0.02      0.97      0.05      1696

    accuracy                           0.88    554082
   macro avg       0.51      0.92      0.49    554082
weighted avg       1.00      0.88      0.93    554082



**Exercise**

What do you observe about the precision/recall of the fraud class? 

<details><summary>Click here for answer</summary>
    
It seems that by placing more weights on 'fraud' class, we have managed to improve the recall rate for 'Fraud' but also increase the false positive. 

### Oversampling 

**TO BE DONE**

#### Non-linear Model

It seems that our linear model is not able to classify fraud/non-fraud very well. Let's try a more complex ensemble model with boosting algorithms, in this case we will use a very fast boosting algorithm called lightGBM

Note: We did not cover this algorithm in the lecture. but we are using it here for comparison only. To learn more about lightGBM, you can refer to the [lightGBM website](https://github.com/microsoft/LightGBM) 

In [83]:
import lightgbm as lgb

lgbm_clf = lgb.LGBMClassifier(num_leaves=30, learning_rate=0.05, n_estimators=30) 
lgbm_clf.fit(X_train, y_train)


[LightGBM] [Info] Number of positive: 6517, number of negative: 2209810
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.065352 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2042
[LightGBM] [Info] Number of data points in the train set: 2216327, number of used features: 9
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.002940 -> initscore=-5.826248
[LightGBM] [Info] Start training from score -5.826248


LGBMClassifier(learning_rate=0.05, n_estimators=30, num_leaves=30)

In [84]:
train_probs = lgbm_clf.predict_proba(X_train)
print('AUPRC for train set = {}'.format(average_precision_score(y_train, train_probs[:, 1])))
test_probs = lgbm_clf.predict_proba(X_test)
print('AUPRC for test set = {}'.format(average_precision_score(y_test, test_probs[:, 1])))

AUPRC for train set = 0.9984903438921692
AUPRC for test set = 0.997910581279196


In [85]:
test_preds = lgbm_clf.predict(X_test)
print(classification_report(y_test, test_preds))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00    552386
           1       1.00      1.00      1.00      1696

    accuracy                           1.00    554082
   macro avg       1.00      1.00      1.00    554082
weighted avg       1.00      1.00      1.00    554082



We can see that we achieve a almost perfect classifier! 

In [ ]:
probs = lr_clf.predict_proba(testX)
probs.shape

In [ ]:
probs[:, 1]

In [ ]:
scores

In [ ]:
print('AUPRC = {}'.format(average_precision_score(testY, scores)))
print('AUPRC = {}'.format(average_precision_score(testY, probs[:, 1])))

In [ ]:
testY_preds = lr_clf.predict(testX)

from sklearn.metrics import classification_report

print(classification_report(testY, testY_preds))

The model we have learnt has a degree of bias and is slighly underfit. This is indicated by the levelling in AUPRC as the size of the training set is increased in the cross-validation curve below. The easiest way to improve the performance of the model still further is to increase the *max_depth* parameter of the XGBClassifier at the expense of the longer time spent learning the model. Other parameters of the classifier that can be adjusted to correct for the effect of the modest underfitting include decreasing *min_child_weight* and decreasing *reg_lambda*.

In [ ]:
train_size_abs, train_scores, test_scores = learning_curve(\
                LogisticRegression(), trainX, trainY, train_sizes=[0.3, 0.6, 0.9], scoring='average_precision')

In [ ]:
# Long computation in this cell (~6 minutes)
from sklearn.model_selection import learning_curve

trainSizes, trainScores, crossValScores = learning_curve(\
LogisticRegression(penalty='l2', class_weight='balance'), trainX,\
                                         trainY)

In [ ]:
print(train_scores)
print(train_size_abs)

In [ ]:
trainScores = train_scores
trainSizes = train_size_abs
crossValScores = test_scores

In [ ]:
trainScoresMean = np.mean(trainScores, axis=1)
trainScoresStd = np.std(trainScores, axis=1)
crossValScoresMean = np.mean(crossValScores, axis=1)
crossValScoresStd = np.std(crossValScores, axis=1)

colours = plt.cm.tab10(np.linspace(0, 1, 9))

fig = plt.figure(figsize = (14, 9))
plt.fill_between(trainSizes, trainScoresMean - trainScoresStd,
    trainScoresMean + trainScoresStd, alpha=0.1, color=colours[0])
plt.fill_between(trainSizes, crossValScoresMean - crossValScoresStd,
    crossValScoresMean + crossValScoresStd, alpha=0.1, color=colours[1])
plt.plot(trainSizes, trainScores.mean(axis = 1), 'o-', label = 'train', \
         color = colours[0])
plt.plot(trainSizes, crossValScores.mean(axis = 1), 'o-', label = 'cross-val', \
         color = colours[1])

ax = plt.gca()
for axis in ['top','bottom','left','right']:
    ax.spines[axis].set_linewidth(2)

handles, labels = ax.get_legend_handles_labels()
plt.legend(handles, ['train', 'cross-val'], bbox_to_anchor=(0.8, 0.15), \
               loc=2, borderaxespad=0, fontsize = 16);
plt.xlabel('training set size', size = 16); 
plt.ylabel('AUPRC', size = 16)
plt.title('Learning curves indicate slightly underfit model', size = 20);

<a href='#top'>back to top</a>

<a id='conclusion'></a>
#### 7. Conclusion

We thoroughly interrogated the data at the outset to gain insight into which features could be discarded and those which could be valuably engineered. The plots provided visual confirmation that the data could be indeed be discriminated with the aid of the new features. To deal with the large skew in the data, we chose an appropriate metric and used an ML algorithm based on an ensemble of decision trees which works best with strongly imbalanced classes. The method used in this kernel should therefore be broadly applicable to a range of such problems.

*Acknowledgements*: Thanks to Edgar Alonso Lopez-Rojas for posting this dataset.

*Hope you enjoyed reading this kernel as much as I had fun writing it. Please feel free to fork, upvote, and leave your comments to make my day* :-)

<a href='#top'>back to top</a>